# Bitly Data Gathering and Transformation


### Problem Descripition 

In 2012, URL shortening service Bitly partnered with the US government website USA.gov to provide a feed of anonymous data gathered from users who shorten links ending with .gov or .mil.

### Objective:
- Scripting using Semi-Structured Data
- Data Transformation
- Python RegEx

The text file comes in JSON format and here are the important keys and their description. 

|key| description |
|---|-----------|
| a|Denotes information about the web browser and operating system|
| tz | time zone |
| r | URL the user come from |
| u | URL where the user headed to |
| t | Timestamp when the user start using the website in UNIX format |
| hc | Timestamp when user exit the website in UNIX format |
| cy | City from which the request intiated |
| ll | Longitude and Latitude |

In [1]:
# retrieve one instance of the file in a list of dictionaries (as example)
import json
records = [json.loads(line) for line in open('usa.gov_click_data.json')]
# Print the first occurance
records[0]

{'a': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.78 Safari/535.11',
 'c': 'US',
 'nk': 1,
 'tz': 'America/New_York',
 'gr': 'MA',
 'g': 'A6qOVH',
 'h': 'wfLQtf',
 'l': 'orofrog',
 'al': 'en-US,en;q=0.8',
 'hh': '1.usa.gov',
 'r': 'http://www.facebook.com/l/7AQEFzjSi/1.usa.gov/wfLQtf',
 'u': 'http://www.ncbi.nlm.nih.gov/pubmed/22415991',
 't': 1333307030,
 'hc': 1333307037,
 'cy': 'Danvers',
 'll': [42.576698, -70.954903]}

## Steps:

### 1- Write a script can transform the JSON files to a DataFrame 

### 2- Create CSV file having the following columns:
- web_browser
        The web browser that has requested the service
- operating_sys
        operating system that intiated this request
- from_url

        The main URL the user came from

    **In a short format**:
    Ex: `www.facebook.com` Not:  `http://www.facebook.com/l/7AffQEFzjSi/1.usa.gov/wfLQtf`
     
    
- to_url

       The same applied like `to_url`
   
- city

        The city from which the the request was sent
    
- longitude

        The longitude where the request was sent
- latitude

        The latitude where the request was sent

- time_zone
        
        The time zone that the city follow
        
- time_in

        Time when the request started
- time_out
        
        Time when the request is ended
        
        


#### Note: We will make sure that the final dataframes have no NaNs at all.

### Step 1:

In [2]:
# Import libraries 
import numpy as np
import pandas as pd
from datetime import datetime

In [3]:
# Create the DataFrame 
df = pd.read_json('usa.gov_click_data.json', lines=True)
df

,a,c,nk,tz,gr,g,h,l,al,hh,r,u,t,hc,cy,ll
0,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,US,1,America/New_York,MA,A6qOVH,wfLQtf,orofrog,"en-US,en;q=0.8",1.usa.gov,http://www.facebook.com/l/7AQEFzjSi/1.usa.gov/...,http://www.ncbi.nlm.nih.gov/pubmed/22415991,1333307030,1333307037,Danvers,"[42.576698, -70.954903]"
1,GoogleMaps/RochesterNY,US,0,America/Denver,UT,mwszkS,mwszkS,bitly,NaN,j.mp,http://www.AwareMap.com/,http://www.monroecounty.gov/etc/911/rss.php,1331923249,1308262393,Provo,"[40.218102, -111.613297]"
2,Mozilla/4.0 (Windows NT 6.1; MSIE 8.0; Windows...,US,1,America/New_York,DC,xxr3Qb,xxr3Qb,bitly,en-US,1.usa.gov,http://t.co/03elZC4Q,http://boxer.senate.gov/en/press/releases/0316...,1333407030,1333407035,Washington,"[38.9007, -77.043098]"
3,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_6_8)...,BR,0,America/Sao_Paulo,27,zCaLwp,zUtuOu,alelex88,pt-br,1.usa.gov,direct,http://apod.nasa.gov/apod/ap120312.html,1333507030,1333507044,Braz,"[-23.549999, -46.616699]"
4,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,US,0,America/New_York,MA,9b6kNl,9b6kNl,bitly,"en-US,en;q=0.8",bit.ly,http://www.shrewsbury-ma.gov/selco/,http://www.shrewsbury-ma.gov/egov/gallery/1341...,1333607030,1333607039,Shrewsbury,"[42.286499, -71.714699]"
5,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,US,0,America/New_York,MA,axNK8c,axNK8c,bitly,"en-US,en;q=0.8",bit.ly,http://www.shrewsbury-ma.gov/selco/,http://www.shrewsbury-ma.gov/egov/gallery/1341...,1333707030,1333707048,Shrewsbury,"[42.286499, -71.714699]"
6,Mozilla/5.0 (Windows NT 5.1) AppleWebKit/535.1...,PL,0,Europe/Warsaw,77,wcndER,zkpJBR,bnjacobs,"pl-PL,pl;q=0.8,en-US;q=0.6,en;q=0.4",1.usa.gov,http://plus.url.google.com/url?sa=z&n=13319232...,http://www.nasa.gov/mission_pages/nustar/main/...,1333807030,1333807040,Luban,"[51.116699, 15.2833]"
7,Mozilla/5.0 (Windows NT 6.1; rv:2.0.1) Gecko/2...,None,0,,NaN,wcndER,zkpJBR,bnjacobs,"bg,en-us;q=0.7,en;q=0.3",1.usa.gov,http://www.facebook.com/,http://www.nasa.gov/mission_pages/nustar/main/...,1334007036,13340070364,NaN,NaN
8,Opera/9.80 (Ubuntu 14.04.6; Linux zbov; U; en)...,None,0,,NaN,wcndER,zkpJBR,bnjacobs,"en-US, en",1.usa.gov,http://www.facebook.com/l.php?u=http%3A%2F%2F1...,http://www.nasa.gov/mission_pages/nustar/main/...,1333907030,1333907042,NaN,NaN
9,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,None,0,,NaN,zCaLwp,zUtuOu,alelex88,"pt-BR,pt;q=0.8,en-US;q=0.6,en;q=0.4",1.usa.gov,http://t.co/o1Pd0WeV,http://apod.nasa.gov/apod/ap120312.html,1334007030,1334007040,NaN,NaN


In [4]:
# Use only important keys 
df = df[['a', 'tz' ,'r' , 'u' , 't', 'hc', 'cy', 'll']]
df.head()

,a,tz,r,u,t,hc,cy,ll
0,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,America/New_York,http://www.facebook.com/l/7AQEFzjSi/1.usa.gov/...,http://www.ncbi.nlm.nih.gov/pubmed/22415991,1333307030,1333307037,Danvers,"[42.576698, -70.954903]"
1,GoogleMaps/RochesterNY,America/Denver,http://www.AwareMap.com/,http://www.monroecounty.gov/etc/911/rss.php,1331923249,1308262393,Provo,"[40.218102, -111.613297]"
2,Mozilla/4.0 (Windows NT 6.1; MSIE 8.0; Windows...,America/New_York,http://t.co/03elZC4Q,http://boxer.senate.gov/en/press/releases/0316...,1333407030,1333407035,Washington,"[38.9007, -77.043098]"
3,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_6_8)...,America/Sao_Paulo,direct,http://apod.nasa.gov/apod/ap120312.html,1333507030,1333507044,Braz,"[-23.549999, -46.616699]"
4,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,America/New_York,http://www.shrewsbury-ma.gov/selco/,http://www.shrewsbury-ma.gov/egov/gallery/1341...,1333607030,1333607039,Shrewsbury,"[42.286499, -71.714699]"


### Step 2:
- web browser

In [5]:
# web_browser
pattern = r'(\w+)'

# Create new DataFrame 
df2 = pd.DataFrame()

# Insert the new coulomn in df2
df2['web_browser'] = df['a'].str.extract(pattern)
df2

,web_browser
0,Mozilla
1,GoogleMaps
2,Mozilla
3,Mozilla
4,Mozilla
5,Mozilla
6,Mozilla
7,Mozilla
8,Opera
9,Mozilla


- Operating system 

In [6]:
# operating_sys
pattern = r"(Windows|Macintosh|Google|Linux)"

# Insert the new coulomn in df2
df2['operating_sys'] = df['a'].str.extract(pattern)
df2

,web_browser,operating_sys
0,Mozilla,Windows
1,GoogleMaps,Google
2,Mozilla,Windows
3,Mozilla,Macintosh
4,Mozilla,Windows
5,Mozilla,Windows
6,Mozilla,Windows
7,Mozilla,Windows
8,Opera,Linux
9,Mozilla,Windows


-  From URL

In [7]:
# from_url
pattern = r"(//.*com/|//.*net/|//.*gov/|//.*/)"

w = df['r'].str.extract(pattern)

# Insert the new coulomn in df2
df2['from_url'] = w[0].str[2:-1]
df2['from_url'] = df2['from_url'].fillna('direct')
df2

,web_browser,operating_sys,from_url
0,Mozilla,Windows,www.facebook.com
1,GoogleMaps,Google,www.AwareMap.com
2,Mozilla,Windows,t.co
3,Mozilla,Macintosh,direct
4,Mozilla,Windows,www.shrewsbury-ma.gov
5,Mozilla,Windows,www.shrewsbury-ma.gov
6,Mozilla,Windows,plus.url.google.com
7,Mozilla,Windows,www.facebook.com
8,Opera,Linux,www.facebook.com
9,Mozilla,Windows,t.co


-  To URL

In [8]:
# to_url
pattern = r"(//.*com/|//.*net/|//.*gov/|//.*/)"

w = df['u'].str.extract(pattern)

# Insert the new coulomn in df2
df2['to_url'] = w[0].str[2:-1]
df2['to_url'] = df2['to_url'].fillna('direct')
df2

,web_browser,operating_sys,from_url,to_url
0,Mozilla,Windows,www.facebook.com,www.ncbi.nlm.nih.gov
1,GoogleMaps,Google,www.AwareMap.com,www.monroecounty.gov
2,Mozilla,Windows,t.co,boxer.senate.gov
3,Mozilla,Macintosh,direct,apod.nasa.gov
4,Mozilla,Windows,www.shrewsbury-ma.gov,www.shrewsbury-ma.gov/egov
5,Mozilla,Windows,www.shrewsbury-ma.gov,www.shrewsbury-ma.gov/egov
6,Mozilla,Windows,plus.url.google.com,www.nasa.gov
7,Mozilla,Windows,www.facebook.com,www.nasa.gov
8,Opera,Linux,www.facebook.com,www.nasa.gov
9,Mozilla,Windows,t.co,apod.nasa.gov


-  City

In [9]:
# city

df2['city'] = df['cy']
df2['city'] = df2['city'].fillna('Unknown')
df2

,web_browser,operating_sys,from_url,to_url,city
0,Mozilla,Windows,www.facebook.com,www.ncbi.nlm.nih.gov,Danvers
1,GoogleMaps,Google,www.AwareMap.com,www.monroecounty.gov,Provo
2,Mozilla,Windows,t.co,boxer.senate.gov,Washington
3,Mozilla,Macintosh,direct,apod.nasa.gov,Braz
4,Mozilla,Windows,www.shrewsbury-ma.gov,www.shrewsbury-ma.gov/egov,Shrewsbury
5,Mozilla,Windows,www.shrewsbury-ma.gov,www.shrewsbury-ma.gov/egov,Shrewsbury
6,Mozilla,Windows,plus.url.google.com,www.nasa.gov,Luban
7,Mozilla,Windows,www.facebook.com,www.nasa.gov,Unknown
8,Opera,Linux,www.facebook.com,www.nasa.gov,Unknown
9,Mozilla,Windows,t.co,apod.nasa.gov,Unknown


-  Longitude and Latitude

In [10]:
# longitude, latitude

df2['longitude'] = list(df.ll.str)[0]
df2['latitude'] = list(df.ll.str)[1]
df2 = df2.fillna('Unknown')
df2

C:\Users\Andrew\AppData\Local\Temp/ipykernel_22768/1246388268.py:3: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  df2['longitude'] = list(df.ll.str)[0]
C:\Users\Andrew\AppData\Local\Temp/ipykernel_22768/1246388268.py:4: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  df2['latitude'] = list(df.ll.str)[1]


,web_browser,operating_sys,from_url,to_url,city,longitude,latitude
0,Mozilla,Windows,www.facebook.com,www.ncbi.nlm.nih.gov,Danvers,42.576698,-70.954903
1,GoogleMaps,Google,www.AwareMap.com,www.monroecounty.gov,Provo,40.218102,-111.613297
2,Mozilla,Windows,t.co,boxer.senate.gov,Washington,38.9007,-77.043098
3,Mozilla,Macintosh,direct,apod.nasa.gov,Braz,-23.549999,-46.616699
4,Mozilla,Windows,www.shrewsbury-ma.gov,www.shrewsbury-ma.gov/egov,Shrewsbury,42.286499,-71.714699
5,Mozilla,Windows,www.shrewsbury-ma.gov,www.shrewsbury-ma.gov/egov,Shrewsbury,42.286499,-71.714699
6,Mozilla,Windows,plus.url.google.com,www.nasa.gov,Luban,51.116699,15.2833
7,Mozilla,Windows,www.facebook.com,www.nasa.gov,Unknown,Unknown,Unknown
8,Opera,Linux,www.facebook.com,www.nasa.gov,Unknown,Unknown,Unknown
9,Mozilla,Windows,t.co,apod.nasa.gov,Unknown,Unknown,Unknown


-  Time zone

In [11]:
# time_zone

df2['time_zone'] = df['tz']
df2[df2 == ''] = 'Unknown'

df2

,web_browser,operating_sys,from_url,to_url,city,longitude,latitude,time_zone
0,Mozilla,Windows,www.facebook.com,www.ncbi.nlm.nih.gov,Danvers,42.576698,-70.954903,America/New_York
1,GoogleMaps,Google,www.AwareMap.com,www.monroecounty.gov,Provo,40.218102,-111.613297,America/Denver
2,Mozilla,Windows,t.co,boxer.senate.gov,Washington,38.9007,-77.043098,America/New_York
3,Mozilla,Macintosh,direct,apod.nasa.gov,Braz,-23.549999,-46.616699,America/Sao_Paulo
4,Mozilla,Windows,www.shrewsbury-ma.gov,www.shrewsbury-ma.gov/egov,Shrewsbury,42.286499,-71.714699,America/New_York
5,Mozilla,Windows,www.shrewsbury-ma.gov,www.shrewsbury-ma.gov/egov,Shrewsbury,42.286499,-71.714699,America/New_York
6,Mozilla,Windows,plus.url.google.com,www.nasa.gov,Luban,51.116699,15.2833,Europe/Warsaw
7,Mozilla,Windows,www.facebook.com,www.nasa.gov,Unknown,Unknown,Unknown,Unknown
8,Opera,Linux,www.facebook.com,www.nasa.gov,Unknown,Unknown,Unknown,Unknown
9,Mozilla,Windows,t.co,apod.nasa.gov,Unknown,Unknown,Unknown,Unknown


-   Time in, Time out

In [12]:
# time_in, time_out

df2['time_in'] = df["t"].apply(lambda x: datetime.fromtimestamp(x))
df2['time_out'] = df["hc"].apply(lambda x: datetime.fromtimestamp(x))

df2

,web_browser,operating_sys,from_url,to_url,city,longitude,latitude,time_zone,time_in,time_out
0,Mozilla,Windows,www.facebook.com,www.ncbi.nlm.nih.gov,Danvers,42.576698,-70.954903,America/New_York,2012-04-01 21:03:50,2012-04-01 21:03:57
1,GoogleMaps,Google,www.AwareMap.com,www.monroecounty.gov,Provo,40.218102,-111.613297,America/Denver,2012-03-16 20:40:49,2011-06-17 00:13:13
2,Mozilla,Windows,t.co,boxer.senate.gov,Washington,38.9007,-77.043098,America/New_York,2012-04-03 00:50:30,2012-04-03 00:50:35
3,Mozilla,Macintosh,direct,apod.nasa.gov,Braz,-23.549999,-46.616699,America/Sao_Paulo,2012-04-04 04:37:10,2012-04-04 04:37:24
4,Mozilla,Windows,www.shrewsbury-ma.gov,www.shrewsbury-ma.gov/egov,Shrewsbury,42.286499,-71.714699,America/New_York,2012-04-05 08:23:50,2012-04-05 08:23:59
5,Mozilla,Windows,www.shrewsbury-ma.gov,www.shrewsbury-ma.gov/egov,Shrewsbury,42.286499,-71.714699,America/New_York,2012-04-06 12:10:30,2012-04-06 12:10:48
6,Mozilla,Windows,plus.url.google.com,www.nasa.gov,Luban,51.116699,15.2833,Europe/Warsaw,2012-04-07 15:57:10,2012-04-07 15:57:20
7,Mozilla,Windows,www.facebook.com,www.nasa.gov,Unknown,Unknown,Unknown,Unknown,2012-04-09 23:30:36,2392-09-24 01:06:04
8,Opera,Linux,www.facebook.com,www.nasa.gov,Unknown,Unknown,Unknown,Unknown,2012-04-08 19:43:50,2012-04-08 19:44:02
9,Mozilla,Windows,t.co,apod.nasa.gov,Unknown,Unknown,Unknown,Unknown,2012-04-09 23:30:30,2012-04-09 23:30:40


In [13]:
# Saving the DataFrame 
df2.to_csv('usa.gov_click_data.csv', index=False)